In [2]:
import csv, os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
All_data = []
for i in range(10):
    All_data.append(pd.read_csv(f'../../preprocess_data/All_data_part{i}.csv', index_col=0, low_memory=False))
    # print(len(All_data[i].columns))
    
All= pd.concat(All_data)

In [6]:
import re
def get_num(x):
    matchs = re.findall(r"(\d+\.?\d*)\s*mg/kg",x)
    print(matchs)
    if len(matchs) >= 1:
        matchs = [float(i) for i in matchs]
        return matchs
    else:
        return None

In [7]:
def mg_select(value):
    if type(value) == str:
        if 'mg/kg' in value:
            return get_num(value)
        else:
            return None
    elif type(value) == list:
        lis = []
        for i in value:
            if 'mg/kg' in i:
                lis.extend(get_num(i))
        if len(lis) > 0:
            return lis
        else:
            return None

    return None

temp['dose'] = temp['arm_group/description'].apply(mg_select)

['90', '120', '5', '10', '90', '5']
['60', '90', '5', '5']
['5', '3', '5', '5', '5', '3', '5', '5']
['5', '5']
['50', '12', '40']
['30', '60', '60', '60', '60', '60', '60', '60', '60', '60', '60', '60', '60', '60']
['2', '4', '6', '8', '8']
['1']
['40', '12', '40', '12']
['40']
['2']
['5.0']
['0.14', '0.14']
['60', '60']
['1']
['5']
['10', '03']
['15', '2.5']
['500', '750']
['500']
['4', '2', '8', '6', '4', '2', '8', '6']
['3']
['60', '100']
['6']
['40', '1', '5']
['0.15']
['2', '1.5']
['0.1', '0.15', '0.20', '0.25', '0.30', '1']
['4', '2', '6', '4', '2', '6']
['4', '2', '4', '2']
['0.15']
['15']
['0.24']
['0.01', '0.04', '0.16', '0.32']
['8', '12']
['5', '8.5']
['10', '5', '10', '5', '3']
['0.3']
['4', '2']
['10', '1.5']
['5', '5']
['100', '100']
['4', '2', '5']
['12', '12', '20', '35', '6', '12', '30', '40', '12', '12', '20', '35', '6', '12', '30', '40', '12', '12', '20', '35', '6', '12', '30', '40', '12', '12', '20', '35', '6', '12', '30', '40']
['0.04', '0.08', '0.12', '0.16']
['0.

In [44]:
All['intervention/intervention_type'].value_counts()

intervention/intervention_type
Drug                                                                      61032
['Drug', 'Drug']                                                          57942
Other                                                                     42591
Device                                                                    32324
Behavioral                                                                23488
                                                                          ...  
['Procedure', 'Procedure', 'Procedure', 'Procedure', 'Drug', 'Device']        1
['Drug', 'Drug', 'Drug', 'Drug', 'Drug', 'Behavioral']                        1
['Procedure', 'Biological', 'Drug', 'Procedure']                              1
['Device', 'Behavioral', 'Other', 'Device']                                   1
['Device', 'Diagnostic Test', 'Device', 'Device']                             1
Name: count, Length: 6889, dtype: int64

In [11]:
(All['intervention/intervention_type'][All['dose'].notna()] == 'Drug').sum()

2361

In [38]:
temp = All[['intervention/intervention_type', 'dose', 'arm_group/description']][All['dose'].notna()]

In [41]:
temp = temp[temp['intervention/intervention_type'] == 'Drug']

In [33]:
temp = temp[(temp['intervention/intervention_type'] == 'Drug')]

In [42]:
temp

,intervention/intervention_type,dose,arm_group/description
NCT00000142,Drug,"[5.0, 3.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0]",['IV (in the vein) treatment deferred until re...
NCT00001962,Drug,[1.0],"daclizumab, 1 mg/kg, will be given for a total..."
NCT00005594,Drug,[6.0],All patients will begin treatment at a dose of...
NCT00008697,Drug,"[0.1, 0.15, 0.2, 0.25, 0.3, 1.0]",['Cohort 1 Arsenic trioxide = 0.1 mg/kg/day x ...
NCT00025883,Drug,[0.24],subcutaneous metreleptin injections in one to ...
...,...,...,...
NCT06256211,Drug,"[15.0, 25.0, 15.0, 15.0, 7.5]",['Intravenous paracetamol dose 15 mg/kg body w...
NCT06257316,Drug,"[1.2, 1.2, 1.2, 1.2, 1.2, 1.2]",['If the mean arterial pressure decreases by m...
NCT06260774,Drug,"[0.4, 0.8, 3.2]","['0.4 mg/kg of TTX-MC138', '0.8 mg/kg of TTX-M..."
NCT06261996,Drug,"[1.0, 0.5]",['Fospropofol is administered to patients afte...


In [43]:
maxedge = 0
for i in temp['dose'].dropna().index:
    if temp['dose'][i] == False:
        continue
    if len(temp['dose'][i]) == 1:
        continue
    else:
        if (max(temp['dose'][i])- min(temp['dose'][i])) > maxedge:
            maxedge = max(temp['dose'][i])- min(temp['dose'][i])
            print(temp['arm_group/description'][i])

['IV (in the vein) treatment deferred until retinitis progressed, either:\r\n5 mg/kg IV (in the vein) of body weight once weekly for two weeks, then maintenance therapy with cidofovir, 3mg/kg once every 2 weeks, or\r\n5mg/kg IV (in the vein) once weekly for 2 weeks, then maintenance therapy with cidofovir, 5mg/kg once every two weeks.', '5 mg/kg IV (in the vein) of body weight once weekly for two weeks, then maintenance therapy with cidofovir, 3mg/kg once every 2 weeks', '5mg/kg IV (in the vein) once weekly for 2 weeks, then maintenance therapy with cidofovir, 5mg/kg once every two weeks.']
All patients enrolled received daily doses of subcutaneous injection of increased doses of anakinra starting at 0.5mg/kg/day up to a maximum 10mg/kg/day to achieve disease remission.
['Subjects with moderate hepatic impairment as defined by a Child-Pugh score of 7-9 will be included. Subjects will be given GR270773 as a loading infusion of 25 milligram per kilogram per hour (mg/kg/hr) for 2 hours fo

In [59]:
maxedge = 0
for i in dose:
    if i == False:
        continue
    if len(i) == 1:
        continue
    else:
        if (max(i)- min(i)) > maxedge:
            maxedge = max(i)- min(i)
            print(i)

AttributeError: 'bool' object has no attribute 'index'

In [14]:
All['arm_group/description'][All['arm_group/description'].apply(mg_select) != False].value_counts()

arm_group/description
['the control group will be given the same volume of saline as the experimental group', 'the experimental group will be given l-1.5mg lidocaine and then 2mg/kg/h']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              4
Geptanolimab Injection, 3mg/kg every 2 weeks                                                                                                                                                                                                                                                   

In [32]:
All['location/facility/address/city'].value_counts()

location/facility/address/city
Seoul                                                                                                                                                                                                                                                                6062
New York                                                                                                                                                                                                                                                             5825
Beijing                                                                                                                                                                                                                                                              5714
Boston                                                                                                                                                                     

In [5]:
def change_description(value):
    if type(value) == str:
        value = value.lower()
        if 'description' in value:
            return 'description'
    
    return value
    
All.loc[All['why_stopped'].apply(change_description) == 'description','why_stopped'] = All.loc[All['why_stopped'].apply(change_description) == 'description','detailed_description/textblock']

In [6]:
def select_reason(value, reason):
    # 包含字段的就返回True
    for r in reason:
        if r==value or r in value:
            return True
        
    return False

failure reason 分为

poor enrollment: accrual \ enroll \ inclusion

efficacy: efficacy \ ineffective 

safety: toxicity \ adverse event \ death 

others: fundings \ PI left \ COVID-19 \ administrative

In [7]:
why_stopped_label = pd.read_csv('why_stopped_labels.csv', index_col=0)

In [8]:
why_stopped = dict(zip(why_stopped_label.index, why_stopped_label['label']))

In [9]:
PoorEnrollment = ['enroll', 'accrual', 'no participants enrolled', 'insufficient enrollment', 'no participants enrolled', 'poor enrollment rate', 'poor enrolment', 'enrollment difficulties', 
                'difficulty in enrollment', 'no enrollment', 'inadequate enrollment', 'study terminated poor enrollment', 'poor patient enrollment', 'study stopped poor enrollment', 
                'the study was terminated poor enrollment', 'terminated poor enrollment', 'difficulty in recruiting patients', 'no patients enrolled', 'inability to recruit patients',
                'insufficient number of participants', 'poor recrutment', 'poor inclusion', 'no participants', 'because of difficulties in recruting', 'no enrolement', 'due to poor recrutment', 
                'because a significant decrease of viable candidates for the study', 'the study was terminated due to insufficient sample size and protocol compliance issue', 'poor participation',
                'administrative reasons, and did not meet sample size requirements', 'insufficient patients with ckd willing to be injected with heparin', 'no patient enter this study',
                'poor patients', 'number of patients', 'no patients', 'no inclusion', 'difficulties of inclusion', 'inclusion difficulties', 'unrealistic to succeed with sufficient inclusion',
                'inclusion rate not as expected', 'inclusion rate', 'no enough inclusion', 'bad inclusion', 'inclusion criteria difficulties', 'poor new inclusion', 'not able to include enough participants']
efficacy = ['efficacy', 'treatment ineffective', 'poor efficacy', 'no convincing results', 'obtained results were not good. protocol was proved ineffective', 'the study was stopped after the first interim analysis due to futility',
            'the study completed the first phase and did not meet efficacy end point to enter into the\r\n    randomized phase', 
            'the decision to stop the trial was based on efficacy results in the overall schizophrenia\r\n    participant population',
            'futility', ]
safety = ['safety', 'safety concerns', 'toxicity', 'toxicology', 'adverse event', 'due to adverse events', 'increased number of patients death']
Others = ['business', 'funded', ' funds', ' pi ', 'funding', 'no funding', 'funding', 'financial constraints', 'pi left', 'pi leaving', 'pi left institution', 'business reasons', "sponsor's decision",
          'pi left the institution', 'loss of funding', 'insufficient funding', 'funding unavailable', 'administrative', 'covid', 'sponsor terminat', 'poor financial', 'sponsor withdrew',
            'poor funds', 'funding withdrawn', 'funding issues', 'funding ended', 'not funded', 'investigator left institution', 'principal investigator left institution', 'cv-19',
            'study not funded', 'funding stopped', 'strategic business decision', 'pi has left institution', 'business decision', 'funder decision', 'due to poor finace', 'sponsor request',
            'insufficient funds', 'financial issues', 'investigator left', 'sponsor decision', 'microbiological findings of resistance on therapy in patients', 'company decision']

reason = {1:'poor enrollment', 2:'efficacy', 3:'safety', 4:'Others'}
def why_stopped_re(value):
    if type(value) == str:
        value = value.lower()
        value = value.strip().strip('.').replace('accural','accrual').replace('acrual','accrual').replace('recruit', 'enroll').replace('lack of', 'poor').replace('slow', 'low').replace('low', 'poor').replace('enrolment', 'enrollment')
        if 'patient' in value and len(value.split()) < 12: # if the sentence is short and contains patient, it is likely to be poor enrollment
            value = 'poor enrollment' 
            # return 1
        if 'inclusion' in value and len(value.split()) < 12:
            value = 'poor enrollment'
            # return 1
        if 'participant' in value and len(value.split()) < 4:
            value = 'poor enrollment'
            # return 1
        if select_reason(value, Others):
            value = 'Others'
            # return 4
        # if ('enroll' in value) or ('accrual' in value) or ('patient' in value) or ('inclusion' in value) or ('subject' in value) or ('participants' in value):
        #     value = 'poor enrollment'
        if select_reason(value, PoorEnrollment):
            value = 'poor enrollment'
            # return 1
        if select_reason(value, efficacy):
            value = 'efficacy'
            # return 2
        if select_reason(value, safety):
            value = 'safety'
            # return 3
        value = ' '.join(value.strip().split())
        if value in why_stopped.keys():
            value = why_stopped[value]
            if value in reason.keys():
                value = reason[value]
            else:
                value = 'Others'
            
    return value

temp = All['why_stopped'].apply(why_stopped_re)

In [11]:
All = All[All['why_stopped'].notna()]
print("Is NA", temp.isna().sum())
print(All['why_stopped'].value_counts())

Is NA 445592
why_stopped
\n    Lack of funding\r\n                                                                                     242
\n    Slow accrual\r\n                                                                                        233
\n    Sponsor decision\r\n                                                                                    208
\n    Lack of enrollment\r\n                                                                                  157
\n    Low accrual\r\n                                                                                         135
                                                                                                             ... 
\n    The study was stopped due to other research priorities.\r\n                                               1
\n    it became clear that recruitment was infeasible\r\n                                                       1
\n    Sponsor determined this trial does not support their anti

Trial Failure Reason Prediction Output

In [33]:
temp = temp.dropna()

In [34]:
temp.to_csv('trial-failure-reason-prediction/output.csv')

Combine trialOutcomev1.csv and why_stopped

In [27]:
why_stopped = pd.read_csv('trial-failure-reason-prediction/output.csv', index_col=0)
trialOutcome = pd.read_csv('data/IQVIA/trial_outcomes_v1.csv', index_col=0)

In [12]:
print(trialOutcome.index)

Index(['NCT00000102', 'NCT00000105', 'NCT00000172', 'NCT00000173',
       'NCT00000174', 'NCT00000175', 'NCT00000340', 'NCT00000372',
       'NCT00000390', 'NCT00000419',
       ...
       'NCT03729284', 'NCT03729635', 'NCT03731364', 'NCT03731793',
       'NCT03732534', 'NCT03733301', 'NCT03733574', 'NCT03735446',
       'NCT03736044', 'NCT03736473'],
      dtype='object', name='studyid', length=51291)


In [29]:
intersection = why_stopped.index.intersection(trialOutcome.index)
union = why_stopped.index.union(trialOutcome.index)
print(len(intersection), len(union))

9434 79504


In [14]:
# trialOutcome 中的类别怎么转到四个类别？
trialOutcome['trialOutcome'].value_counts()

trialOutcome
Completed, Positive outcome/primary endpoint(s) met          16067
Completed, Outcome unknown                                    9162
Completed, Outcome indeterminate                              4478
Terminated, Poor enrollment                                   4088
Completed, Negative outcome/primary endpoint(s) not met       3767
Terminated, Unknown                                           3103
Terminated, Planned but never initiated                       2562
Terminated, Other                                             2000
Terminated, Lack of efficacy                                  1682
Terminated, Business decision - Other                         1448
Terminated, Safety/adverse effects                             921
Terminated, Lack of funding                                    752
Terminated, Business decision - Pipeline reprioritization      691
Terminated, Business decision - Drug strategy shift            315
Completed, Early positive outcome                

In [28]:
reason = {1:'poor enrollment', 2:'efficacy', 3:'safety', 4:'Others'}
def map2fourcat(x):
    if 'Completed' in x:
        return 'Completed'
    elif 'Poor enrollment' in x:
        return reason[1]
    elif 'Safety/adverse effects' in x:
        return reason[3]
    elif 'Lack of efficacy' in x:
        return reason[2]
    elif 'Negative outcome/primary endpoint(s) not met' in x:
        return reason[2]
    else:
        return reason[4]

trialOutcome['why_stopped'] = trialOutcome['trialOutcome'].apply(map2fourcat)

In [20]:
intersec = pd.merge(trialOutcome.loc[intersection], why_stopped.loc[intersection], right_index=True, left_index=True)
intersec

,trialOutcome,why_stopped_x,why_stopped_y
NCT00000105,"Terminated, Other",Others,Others
NCT00000340,"Terminated, Lack of funding",Others,Others
NCT00000372,"Terminated, Other",Others,safety
NCT00000577,"Terminated, Lack of efficacy",efficacy,Others
NCT00001703,"Terminated, Other",Others,poor enrollment
...,...,...,...
NCT03726281,"Terminated, Business decision - Other",Others,Others
NCT03729284,"Terminated, Business decision - Other",Others,Others
NCT03729284,"Terminated, Planned but never initiated",Others,Others
NCT03732534,"Terminated, Lack of efficacy",efficacy,safety


In [30]:
difference = []
for i in intersection:
    if len(trialOutcome.loc[i].shape) > 1:
        # print(len(trialOutcome.loc[i].shape))
        continue
    if trialOutcome['why_stopped'][i] != why_stopped['why_stopped'][i]:
        print(i, trialOutcome['why_stopped'][i], why_stopped['why_stopped'][i], All['why_stopped'][i])
        difference.append(i)


NCT00000372 Others safety 
    Pairing D-Cycloserine with Clozapine was found to worsen negative side effects in patients with
    Schizophrenia, so the study was suspended.
  
NCT00000577 efficacy Others 
    Record is an ACRN grant summary & not reflective of an individual trial. All ACTs conducted by
    ACRN were individually registered on the PRS.
  
NCT00003454 Others poor enrollment 
    slow accrual
  
NCT00003645 Completed Others 
    Per PI request
  
NCT00003715 Others poor enrollment 
    Terminated: recruiting or enrolling participants has halted prematurely and will not resume;
    participants are no longer being examined or treated
  
NCT00032643 safety Others 
    Funding support withdrawn (much slower than expected enrollment)
  
NCT00037622 Others safety 
    Safety concerns from other studies.
  
NCT00038870 Others poor enrollment 
    Slow accrual.
  
NCT00047554 Completed Others 
    Study objectives met
  
NCT00048087 Others poor enrollment 
    Slow accrual.
  


In [34]:
from collections import Counter
total = 0
for k, v in dict(Counter(trialOutcome.index)).items():
    if v > 1:
        print(k, v, trialOutcome.loc[k]['trialOutcome'])
        total+=1

print(total)

NCT00001094 2 studyid
NCT00001094    Terminated, Planned but never initiated
NCT00001094                        Terminated, Unknown
Name: trialOutcome, dtype: object
NCT00001703 2 studyid
NCT00001703              Terminated, Other
NCT00001703    Terminated, Poor enrollment
Name: trialOutcome, dtype: object
NCT00002664 2 studyid
NCT00002664                          Terminated, Other
NCT00002664    Terminated, Planned but never initiated
Name: trialOutcome, dtype: object
NCT00003172 2 studyid
NCT00003172          Terminated, Lack of efficacy
NCT00003172    Terminated, Safety/adverse effects
Name: trialOutcome, dtype: object
NCT00003205 2 studyid
NCT00003205    Terminated, Business decision - Pipeline repri...
NCT00003205                         Terminated, Lack of efficacy
Name: trialOutcome, dtype: object
NCT00003215 2 studyid
NCT00003215          Terminated, Lack of efficacy
NCT00003215    Terminated, Safety/adverse effects
Name: trialOutcome, dtype: object
NCT00003276 2 studyid
NCT000

如果是hint和gpt有冲突，选择gpt的

In [44]:
union = set(list(union))
indexs = []
result = []
for i in list(union):
    if i in intersection:
        result.append(why_stopped['why_stopped'][i])
        indexs.append(i)
    elif i in why_stopped.index:
        result.append(why_stopped['why_stopped'][i])
        indexs.append(i)
    elif i in trialOutcome.index:
        if len(trialOutcome.loc[i].shape) > 1:
            # print(len(trialOutcome.loc[i].shape))
            continue
        result.append(trialOutcome['why_stopped'][i])
        indexs.append(i)



In [47]:
failure_reason = pd.DataFrame({'nctid': indexs, 'failure_reason': result})
failure_reason[failure_reason['failure_reason'] != 'Completed'].set_index('nctid').to_csv('trial-failure-reason-prediction/output.csv')

In [49]:
set(failure_reason[failure_reason['failure_reason'] != 'Completed']['failure_reason'])

{'Others', 'efficacy', 'poor enrollment', 'safety'}

trial-approval-prediction output

In [54]:
reason = {1:'poor enrollment', 2:'efficacy', 3:'safety', 4:'Others'}
def map2approval(x):
    if x =='Completed, Early positive outcome' or x == 'Completed, Positive outcome/primary endpoint(s) met':
        return 1
    elif 'Poor enrollment' in x:
        return 0
    elif 'Safety/adverse effects' in x:
        return 0
    elif 'Lack of efficacy' in x:
        return 0
    elif 'Negative outcome/primary endpoint(s) not met' in x:
        return 0
    else:
        return -1

trialOutcome['approval'] = trialOutcome['trialOutcome'].apply(map2approval)

In [51]:
approval = why_stopped[why_stopped['why_stopped'] != 'Others']

In [52]:
intersection = approval.index.intersection(trialOutcome.index)
union = approval.index.union(trialOutcome.index)
print(len(intersection), len(union))

5509 63513


In [56]:
union = set(list(union))
indexs = []
result = []
for i in list(union):
    if i in intersection:
        result.append(0)
        indexs.append(i)
    elif i in why_stopped.index:
        result.append(0)
        indexs.append(i)
    elif i in trialOutcome.index:
        if len(trialOutcome.loc[i].shape) > 1:
            # print(len(trialOutcome.loc[i].shape))
            continue
        elif trialOutcome['approval'][i] != -1:
            result.append(trialOutcome['approval'][i])
            indexs.append(i)
        elif trialOutcome['approval'][i] == -1:
            continue



In [57]:
approval = pd.DataFrame({'nctid': indexs, 'approval': result})
approval[approval['approval'] != -1].set_index('nctid').to_csv('trial-approval-prediction/output.csv')

In [58]:
approval['approval'].value_counts()

approval
0    27090
1    16122
Name: count, dtype: int64